In [1]:
pip install psycopg2-binary

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 16.9 MB/s eta 0:00:00


In [2]:
import psycopg2

conn = psycopg2.connect(
host = "apprentice-training-2023-rds.cth7tqaptja4.us-west-1.rds.amazonaws.com",
database = "postgres",
user = "postgres",
password = "hello123")
print('Connected to database')


cursor = conn.cursor()
table_name = "etl_utsav_stock_data"

Connected to database


In [3]:
query = """
    CREATE TABLE IF NOT EXISTS etl_utsav_stock_data (
        identifier VARCHAR(255),
        open VARCHAR(255),
        dayHigh VARCHAR(255),
        dayLow VARCHAR(255),
        lastPrice VARCHAR(255),
        previousClose VARCHAR(255),
        change VARCHAR(255),
        pChange VARCHAR(255),
        yearHigh VARCHAR(255),
        yearLow VARCHAR(255),
        totalTradedValue VARCHAR(255),
        lastUpdateTime VARCHAR(255),
        perChange365d VARCHAR(255),
        perChange30d VARCHAR(255)
    );
    """
cursor.execute(query)
conn.commit()

In [4]:
import urllib3
import json
import pandas as pd

data_json = dict()

# def lambda_handler(event, context):
#     # Fetch data from the API
#     http = urllib3.PoolManager()
#     url = "https://latest-stock-price.p.rapidapi.com/any"
#     headers = {
#         'X-RapidAPI-Key': "da39041ecamshf75624bcf3c0396p1b72b7jsn559d324f9f25",
#         'X-RapidAPI-Host': "latest-stock-price.p.rapidapi.com"
#     }
#     response = http.request('GET', url, headers=headers)
#     data = response.data.decode("utf-8")
#     data_json = json.loads(data)

http = urllib3.PoolManager()
url = "https://latest-stock-price.p.rapidapi.com/any"
headers = {
    'X-RapidAPI-Key': "da39041ecamshf75624bcf3c0396p1b72b7jsn559d324f9f25",
    'X-RapidAPI-Host': "latest-stock-price.p.rapidapi.com"
}

response = http.request('GET', url, headers=headers)
data = response.data.decode("utf-8")
data_json = json.loads(data)
data_json

[]

In [12]:
data_json = pd.read_json('./stock_data.json')

In [14]:
df = pd.DataFrame(data_json)
df

,symbol,identifier,open,dayHigh,dayLow,lastPrice,previousClose,change,pChange,yearHigh,yearLow,totalTradedVolume,totalTradedValue,lastUpdateTime,perChange365d,perChange30d
0,NIFTY SMLCAP 50,NIFTY SMLCAP 50,5349.95,5382.30,5348.35,5378.05,5331.95,46.10,0.86,5421.20,3984.65,398786307,3.311214e+10,22-Aug-2023 16:00:00,23.47,2.86
1,WELSPUNIND,WELSPUNINDEQN,115.30,123.00,114.95,122.75,114.95,7.80,6.79,123.00,62.00,14501926,1.753863e+09,22-Aug-2023 15:59:57,47.28,14.95
2,HINDCOPPER,HINDCOPPEREQN,137.50,146.35,137.10,144.75,136.70,8.05,5.89,160.70,93.85,13124521,1.869194e+09,22-Aug-2023 15:58:50,21.03,13.96
3,BIRLACORPN,BIRLACORPNEQN,1132.50,1192.85,1130.95,1185.45,1126.65,58.80,5.22,1268.00,843.15,281830,3.275175e+08,22-Aug-2023 15:59:28,13.9,-6.04
4,SUZLON,SUZLONEQN,19.95,20.85,19.95,20.85,19.90,0.95,4.77,21.25,6.60,189624524,3.919539e+09,22-Aug-2023 15:59:59,158.44,1.53
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
805,NIFTY100 ESG,NIFTY100 ESG,3644.40,3648.20,3637.10,3642.30,3637.10,5.20,0.14,0.00,0.00,372458187,2.232616e+11,22-Aug-2023 16:00:00,3.84,-0.89
806,NIFTY MNC,NIFTY MNC,21855.85,21897.55,21815.35,21862.10,21797.45,64.65,0.30,22572.55,18651.05,24479745,2.305973e+10,22-Aug-2023 16:00:00,11.7,-1.68
807,NIFTY MID LIQ 15,NIFTY MID LIQ 15,8957.90,9018.45,8934.05,9013.55,8939.80,73.75,0.82,9024.75,7039.85,49532384,2.061848e+10,22-Aug-2023 16:00:00,17.1,2.88
808,NIFTY SERV SECTOR,NIFTY SERV SECTOR,24827.95,24847.60,24734.65,24756.40,24769.10,-12.70,-0.05,25939.50,22333.20,170299427,1.403138e+11,22-Aug-2023 16:00:00,1.34,-2.11


In [16]:
transformed_df.to_csv('cleaned_data.csv')

In [8]:
transformed_df = pd.DataFrame(data_json)
transformed_df.drop(columns=["Volume", 'symbol'], inplace=True, errors="ignore")

In [9]:
transformed_df.columns

Index(['identifier', 'open', 'dayHigh', 'dayLow', 'lastPrice', 'previousClose',
       'change', 'pChange', 'yearHigh', 'yearLow', 'totalTradedVolume',
       'totalTradedValue', 'lastUpdateTime', 'perChange365d', 'perChange30d'],
      dtype='object')

In [11]:
transformed_df

,identifier,open,dayHigh,dayLow,lastPrice,previousClose,change,pChange,yearHigh,yearLow,totalTradedVolume,totalTradedValue,lastUpdateTime,perChange365d,perChange30d
0,NIFTY SMLCAP 50,5349.95,5382.30,5348.35,5378.05,5331.95,46.10,0.86,5421.20,3984.65,398786307,3.311214e+10,22-Aug-2023 16:00:00,23.47,2.86
1,WELSPUNINDEQN,115.30,123.00,114.95,122.75,114.95,7.80,6.79,123.00,62.00,14501926,1.753863e+09,22-Aug-2023 15:59:57,47.28,14.95
2,HINDCOPPEREQN,137.50,146.35,137.10,144.75,136.70,8.05,5.89,160.70,93.85,13124521,1.869194e+09,22-Aug-2023 15:58:50,21.03,13.96
3,BIRLACORPNEQN,1132.50,1192.85,1130.95,1185.45,1126.65,58.80,5.22,1268.00,843.15,281830,3.275175e+08,22-Aug-2023 15:59:28,13.9,-6.04
4,SUZLONEQN,19.95,20.85,19.95,20.85,19.90,0.95,4.77,21.25,6.60,189624524,3.919539e+09,22-Aug-2023 15:59:59,158.44,1.53
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
805,NIFTY100 ESG,3644.40,3648.20,3637.10,3642.30,3637.10,5.20,0.14,0.00,0.00,372458187,2.232616e+11,22-Aug-2023 16:00:00,3.84,-0.89
806,NIFTY MNC,21855.85,21897.55,21815.35,21862.10,21797.45,64.65,0.30,22572.55,18651.05,24479745,2.305973e+10,22-Aug-2023 16:00:00,11.7,-1.68
807,NIFTY MID LIQ 15,8957.90,9018.45,8934.05,9013.55,8939.80,73.75,0.82,9024.75,7039.85,49532384,2.061848e+10,22-Aug-2023 16:00:00,17.1,2.88
808,NIFTY SERV SECTOR,24827.95,24847.60,24734.65,24756.40,24769.10,-12.70,-0.05,25939.50,22333.20,170299427,1.403138e+11,22-Aug-2023 16:00:00,1.34,-2.11


In [10]:
# Update the insert query
insert_query = """
INSERT INTO etl_utsav_stock_data
(identifier, open, dayHigh, dayLow, lastPrice, previousClose, change, pChange, yearHigh, yearLow, totalTradedValue, lastUpdateTime, perChange365d, perChange30d)
VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
"""

# Prepare the data for insertion
data_to_insert = [
    (
        row['identifier'], row['open'], row['dayHigh'], row['dayLow'], row['lastPrice'], row['previousClose'],
        row['change'], row['pChange'], row['yearHigh'], row['yearLow'], row['totalTradedValue'],
        row['lastUpdateTime'], row['perChange365d'], row['perChange30d']
    )
    for index, row in transformed_df.iterrows()
]

# Execute the insert query with the data_to_insert
try:
    cursor.executemany(insert_query, data_to_insert)
    conn.commit()
    print("Data inserted into the database successfully.")
except Exception as e:
    print("Error:", e)
    conn.rollback()  # Roll back the transaction in case of an error
finally:
    cursor.close()
    conn.close()


Data inserted into the database successfully.
